<a href="https://colab.research.google.com/github/ben8169/SWbootcamp_repository/blob/main/2%EC%A3%BC%EC%B0%A8_1%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**꿀벌 종 및 건강상태 분류 프로그램 제출**


성균관대학교/인문과학계열 1학년/김지헌

같이 하고 싶은 팀원 여부 : X (전공생 or 헬스케어 관련 확실한 비전이 있는 팀장님이면 좋겠습니다)

자신있는 분야 : 파이썬 (코딩을 본격적으로 시작한 지 두달 정도 밖에 되지 않았기에, 가장 익숙한 언어이기도 하고, 다양한 함수 및 모듈을 사용하는 방법에 대해 스스로 많이 고민해본 파이썬를 주력으로 최대한 사용하는 것이 최대의 역량을 뽑아낼 수 있는 방법이라 생각하였습니다. 딥러닝의 경우 기초개념부터 실제 활용까지 3주만에 속성으로 배운 터라 부족하지만, 어느정도 해석하고 진행해 나갈 수 있을 정도입니다.)


In [ ]:
import glob
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import numpy as np

In [ ]:
#드라이브 연결 및 압축 해제
!unzip -qq "/content/drive/MyDrive/archive.zip"

In [ ]:
#glob를 통한 파일 확인
output = glob.glob(r'/content/bee_imgs/bee_imgs/*.png')
print(output)

['/content/bee_imgs/bee_imgs/040_093.png', '/content/bee_imgs/bee_imgs/030_830.png', '/content/bee_imgs/bee_imgs/032_771.png', '/content/bee_imgs/bee_imgs/030_429.png', '/content/bee_imgs/bee_imgs/031_002.png', '/content/bee_imgs/bee_imgs/030_720.png', '/content/bee_imgs/bee_imgs/016_163.png', '/content/bee_imgs/bee_imgs/008_186.png', '/content/bee_imgs/bee_imgs/032_826.png', '/content/bee_imgs/bee_imgs/015_992.png', '/content/bee_imgs/bee_imgs/038_391.png', '/content/bee_imgs/bee_imgs/030_869.png', '/content/bee_imgs/bee_imgs/043_070.png', '/content/bee_imgs/bee_imgs/040_031.png', '/content/bee_imgs/bee_imgs/037_018.png', '/content/bee_imgs/bee_imgs/010_443.png', '/content/bee_imgs/bee_imgs/003_195.png', '/content/bee_imgs/bee_imgs/030_780.png', '/content/bee_imgs/bee_imgs/028_014.png', '/content/bee_imgs/bee_imgs/005_263.png', '/content/bee_imgs/bee_imgs/042_004.png', '/content/bee_imgs/bee_imgs/019_814.png', '/content/bee_imgs/bee_imgs/025_017.png', '/content/bee_imgs/bee_imgs/009_0

>**csv 추출**

In [ ]:
# pandas를 통해 csv라는 dataframe 설정
csv = pd.read_csv('/content/bee_data.csv')
csv

,file,date,time,location,zip code,subspecies,health,pollen_carrying,caste
0,041_066.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
1,041_072.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
2,041_073.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
3,041_067.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
4,041_059.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
...,...,...,...,...,...,...,...,...,...
5167,027_011.png,8/20/18,10:03,"San Jose, CA, USA",95124,-1,healthy,True,worker
5168,027_007.png,8/20/18,10:03,"San Jose, CA, USA",95124,-1,healthy,True,worker
5169,027_013.png,8/20/18,10:03,"San Jose, CA, USA",95124,-1,healthy,False,worker
5170,027_012.png,8/20/18,10:03,"San Jose, CA, USA",95124,-1,healthy,False,worker


In [ ]:
#필요한 column만 추출
csv.loc[:, ["file","health","subspecies"]]

,file,health,subspecies
0,041_066.png,hive being robbed,-1
1,041_072.png,hive being robbed,-1
2,041_073.png,hive being robbed,-1
3,041_067.png,hive being robbed,-1
4,041_059.png,hive being robbed,-1
...,...,...,...
5167,027_011.png,healthy,-1
5168,027_007.png,healthy,-1
5169,027_013.png,healthy,-1
5170,027_012.png,healthy,-1


In [ ]:
#Pandas 이용해 category 파악
csv['health'].value_counts()

healthy                       3384
few varrao, hive beetles       579
Varroa, Small Hive Beetles     472
ant problems                   457
hive being robbed              251
missing queen                   29
Name: health, dtype: int64

In [ ]:
csv['subspecies'].value_counts()

Italian honey bee        3008
Russian honey bee         527
Carniolan honey bee       501
1 Mixed local stock 2     472
-1                        428
VSH Italian honey bee     199
Western honey bee          37
Name: subspecies, dtype: int64

>**건강 상태 분류(인덱스 이용)**

In [ ]:
h = csv['health']

In [ ]:
health_num = []
class_list = ['healthy', 'few varrao, hive beetles', 'Varroa, Small Hive Beetles', 'ant problems', 'hive being robbed','missing queen']

for i in h:
  for k in range(len(class_list)):
    if i == class_list[k]:
      health_num.append(k)

In [ ]:
#
train_data = []
files = output
train_data += [(cv2.resize(cv2.cvtColor(cv2.imread(x, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB), (28,28))) for x in files]
train_data = np.array(train_data)

In [ ]:
train_label = health_num
train_label = np.array(train_label).reshape(-1,1)

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same',activation='sigmoid',input_shape=(28, 28, 3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(5172, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data, train_label, epochs=100)

Epoch 1/100


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


162/162 [==============================] - 145s 886ms/step - loss: 1.4241 - accuracy: 0.6311
Epoch 2/100
162/162 [==============================] - 143s 880ms/step - loss: 1.2665 - accuracy: 0.6514
Epoch 3/100
162/162 [==============================] - 121s 747ms/step - loss: 1.1776 - accuracy: 0.6477
Epoch 4/100
162/162 [==============================] - 124s 766ms/step - loss: 1.1178 - accuracy: 0.6630
Epoch 5/100
162/162 [==============================] - 120s 744ms/step - loss: 1.0531 - accuracy: 0.6661
Epoch 6/100
162/162 [==============================] - 120s 743ms/step - loss: 1.0277 - accuracy: 0.6775
Epoch 7/100
162/162 [==============================] - 120s 740ms/step - loss: 0.9918 - accuracy: 0.6868
Epoch 8/100
162/162 [==============================] - 122s 754ms/step - loss: 0.9140 - accuracy: 0.7017
Epoch 9/100
162/162 [==============================] - 120s 744ms/step - loss: 0.8626 - accuracy: 0.7140
Epoch 10/100
162/162 [==============================] - 120s 742ms/

KeyboardInterrupt: ignored

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['accuracy'])

In [ ]:
loss = history.history['loss']
epochs = range(1, len(loss)+1)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.title('Loss')
plt.plot(epochs, history.history['loss'], 'r', label='train_loss')
plt.ylim([0,1])
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='best')

plt.subplot(1, 2, 2)
plt.title('Accuracy')
plt.plot(epochs, history.history['accuracy'], 'b', label='train_accuracy')
plt.ylim([0.5 , 1])
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()